In [3]:
!pip install -q transformers sentencepiece python-docx pandasimport os

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: Could not find a version that satisfies the requirement pandasimport (from versions: none)
ERROR: No matching distribution found for pandasimport


In [2]:
import glob
import os
import pandas as pd
from docx import Document
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import time

# --- モデルとトークナイザーの準備 ---
model_name = "tsmatz/mt5_summarize_japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# GPUが利用可能なら使用
if torch.cuda.is_available():
    model.to('cuda')

def summarize(text: str, target_chars=120):
    """指定文字数で自然な要約を生成"""
    max_length = int(target_chars * 1.3)

    prompt = (
        f"次の文章を、だいたい {target_chars} 文字くらいで自然に要約してください。\n"
        f"【文章】{text}\n"
        "【要約】"
    )

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

    if torch.cuda.is_available():
        inputs = inputs.to('cuda')

    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def read_docx(file_path):
    """Wordファイル (.docx) からテキスト抽出"""
    try:
        document = Document(file_path)
        full_text = "\n".join([p.text.strip() for p in document.paragraphs if p.text.strip()])
        return full_text
    except Exception as e:
        print(f"   Word読み込みエラー {file_path}: {e}")
        return None

def summarize_folder_to_csv(folder_path: str, target_chars: int, output_csv: str):
    """フォルダ内すべての .docx を要約し CSV に保存"""

    if not os.path.isdir(folder_path):
        print(f" エラー: フォルダが存在しません → {folder_path}")
        return

    print(f"---  {folder_path} 内の .docx ファイルを検索 ---")
    file_list = glob.glob(os.path.join(folder_path, "*.docx"))

    if not file_list:
        print(" .docx が見つかりませんでした")
        return

    results = []
    print(f"---  {len(file_list)} 個のファイルを処理します ---")

    for i, file_path in enumerate(file_list):
        start_time = time.time()
        file_name = os.path.basename(file_path)
        print(f"\n[{i+1}/{len(file_list)}] 📄 処理中: {file_name}")

        original_text = read_docx(file_path)

        if original_text and original_text.strip():
            try:
                summary_text = summarize(original_text, target_chars=target_chars)
                actual_length = len(summary_text)
                elapsed_time = time.time() - start_time
                print(f"   → 要約完了 ({actual_length}字 / {elapsed_time:.2f}秒)")

                results.append({
                    "ファイル名": file_name,
                    "元テキスト（100文字）": original_text[:100].replace('\n', ' ') + "...",
                    "希望文字数": target_chars,
                    "実際の文字数": actual_length,
                    "要約結果": summary_text
                })
            except Exception as e:
                print(f"   🚨 要約エラー: {e}")
                results.append({
                    "ファイル名": file_name,
                    "元テキスト（100文字）": "エラー",
                    "希望文字数": target_chars,
                    "実際の文字数": 0,
                    "要約結果": f"要約エラー: {e}"
                })
        else:
            print("    テキストが空のためスキップ")
            results.append({
                "ファイル名": file_name,
                "元テキスト（100文字）": "テキストなし",
                "希望文字数": target_chars,
                "実際の文字数": 0,
                "要約結果": "スキップ"
            })

    # CSV保存
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n---  完了！結果をCSVに保存しました → {output_csv} ---")


# =====================================
# === 大学サーバー用 実行パート ===
# =====================================

# ローカルパスに変更（例）
folder_path = "/home/k101okab/k_data/jiiken5/5-3/PBL/tv"   # ← 自分のフォルダに変更！

print(f"\n--- 🎯 処理対象フォルダ: {folder_path} ---")

# ユーザー入力（または固定値 120）
try:
    target = int(input("要約の文字数（例:120）："))
except ValueError:
    print("無効な入力。120字で実行します。")
    target = 120

timestamp = time.strftime('%Y%m%d_%H%M%S')
output_file = os.path.join(folder_path, f"summary_results_{timestamp}.csv")

summarize_folder_to_csv(folder_path, target_chars=target, output_csv=output_file)


/k_home/k101okab/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at tsmatz/mt5_summarize_japanese and are newly initialized: ['decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- 🎯 処理対象フォルダ: /home/k101okab/k_data/jiiken5/5-3/PBL/tv ---


要約の文字数（例:120）： 100


--- 📁 /home/k101okab/k_data/jiiken5/5-3/PBL/tv 内の .docx ファイルを検索 ---
--- ✅ 74 個のファイルを処理します ---

[1/74] 📄 処理中: 210829.docx
   → 要約完了 (18字 / 1.22秒)

[2/74] 📄 処理中: 220501.docx
   → 要約完了 (51字 / 0.62秒)

[3/74] 📄 処理中: 230528-原稿２.docx
   → 要約完了 (26字 / 0.70秒)

[4/74] 📄 処理中: 200906.docx
   → 要約完了 (31字 / 0.46秒)

[5/74] 📄 処理中: 230917-原稿３.docx
   → 要約完了 (35字 / 0.48秒)

[6/74] 📄 処理中: 220410-もぎナレーション２.docx
   → 要約完了 (14字 / 0.50秒)

[7/74] 📄 処理中: 220109.docx
   → 要約完了 (45字 / 0.44秒)

[8/74] 📄 処理中: 221002-もぎナレ.docx
   → 要約完了 (24字 / 0.40秒)

[9/74] 📄 処理中: 230827-もぎナレ.docx
   → 要約完了 (22字 / 0.66秒)

[10/74] 📄 処理中: 230416-大洲NA案_2稿.docx
   → 要約完了 (35字 / 0.43秒)

[11/74] 📄 処理中: 230402-原稿２.docx
   → 要約完了 (23字 / 0.42秒)

[12/74] 📄 処理中: 230723-原稿０２.docx
   → 要約完了 (10字 / 0.42秒)

[13/74] 📄 処理中: 220710-もぎナレ.docx
   → 要約完了 (50字 / 0.60秒)

[14/74] 📄 処理中: 221009-ナレ原稿２.docx
   → 要約完了 (29字 / 0.36秒)

[15/74] 📄 処理中: 220508-もぎナレ改.docx
   → 要約完了 (43字 / 0.55秒)

[16/74] 📄 処理中: 220605.docx
   → 要約完了 (25字 / 0.56秒)

[17/74] 📄 処理中: 2203